In [1]:
!pip install flask requests transformers

In [16]:
%%writefile app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Initialize sentiment analysis pipeline with distilbert
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Initialize text generation pipeline with google/flan-t5-large.
# Note: T5 is a text-to-text model so we use the "text2text-generation" pipeline.
text_gen_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

def check_sentiment(message):
    result = sentiment_pipeline(message)
    return result[0]  # e.g., {'label': 'NEGATIVE', 'score': 0.99}

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get("message", "").strip()

    # Check sentiment using the distilbert model
    sentiment_result = check_sentiment(user_message)

    # If negative sentiment is detected, return a safe response.
    if sentiment_result['label'] == "NEGATIVE":
        return jsonify({
            "reply": "I sense some anger in your words. Let's try to keep the conversation calm.",
            "sentiment_analysis": sentiment_result
        })

    # Build a prompt that includes the conversation history.
    # For T5, we provide a text-to-text prompt.
    prompt = """
    The following is a conversation between a user and an AI assistant.\n +
    Provide a helpful and polite response.\n
    """

    # Generate a response using google/flan-t5-large
    generated = text_gen_pipeline(prompt, max_length=250, do_sample=True, temperature=0.7)
    full_text = generated[0]['generated_text']

    return jsonify({
        "reply": full_text,
        "sentiment_analysis": sentiment_result,
    })

if __name__ == '__main__':
    # The server will run on 0.0.0.0:5000
    app.run(host='0.0.0.0', port=5000)


Overwriting app.py


In [17]:
# prompt: kill current process listening to port 5000

!fuser -k 5000/tcp


5000/tcp:             2200


In [18]:
!nohup python app.py &


nohup: appending output to 'nohup.out'


In [19]:
import socket
import time

tries = 0

def print_last_lines(filename, num_lines=10):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            total_lines = len(lines)
            start_index = max(0, total_lines - num_lines)
            for line in lines[start_index:]:
                print(line, end="")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")

def check_port(host, port):
  port_closed = True
  time.sleep(1)
  tries = 0
  while port_closed and tries < 15:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.connect((host, port))
            print(f"Port {port} on {host} is open!")
            port_closed = False
        except ConnectionRefusedError:
            print(f"Port {port} on {host} is closed.")
            print_last_lines('nohup.out')
            tries += 1
            time.sleep(5)


check_port("127.0.0.1", 5000)

Port 5000 on 127.0.0.1 is closed.
Port 5000 on 127.0.0.1 is closed.
2025-02-20 01:18:17.776113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740014297.797880    2642 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740014297.804471    2642 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cuda:0
Port 5000 on 127.0.0.1 is open!


In [20]:
import requests

url = "http://127.0.0.1:5000/chat"

# Test messages
messages = [
    "Hello, how are you?",
    "I don't like this at all, it's terrible!",
    "Translate from English to Spanish: I love this course!"
]

for msg in messages:
    payload = {"message": msg}
    response = requests.post(url, json=payload)
    if response.ok:
        result = response.json()
        print(f"User: {msg}")
        print("Chatbot:", result.get("reply"))
        print("Sentiment Analysis:", result.get("sentiment_analysis"))
        if "conversation_history" in result:
            print("Conversation History:")
            for line in result["conversation_history"]:
                print("   " + line)
        print("-" * 50)
    else:
        print("Failed to connect to the chatbot service.")


User: Hello, how are you?
Chatbot: I can't answer that question. I can't find it in the FAQs. What's the problem?
Sentiment Analysis: {'label': 'POSITIVE', 'score': 0.994739830493927}
--------------------------------------------------
User: I don't like this at all, it's terrible!
Chatbot: I sense some anger in your words. Let's try to keep the conversation calm.
Sentiment Analysis: {'label': 'NEGATIVE', 'score': 0.9991625547409058}
--------------------------------------------------
User: Translate from English to Spanish: I love this course!
Chatbot: I'm sorry, I didn't think you'd ask.
Sentiment Analysis: {'label': 'POSITIVE', 'score': 0.999872088432312}
--------------------------------------------------
